<a href="https://colab.research.google.com/github/adrcabral/teste/blob/main/Webscraping_Betexplorer_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Documentação do BeautifullSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc.ptbr/

Site do BetExplorer: https://www.betexplorer.com/soccer

# Instalando o BS4

In [ ]:
!pip install --upgrade beautifulsoup4

# Importando as Bibliotecas

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Pegando Dados da Premier League 2021/2022

In [ ]:
# url = "https://www.betexplorer.com/football/south-africa/premier-league/results/"

nation = "south-africa"
league = "premier-league"
season = "2021-2022"

url = f"https://www.betexplorer.com/soccer/{nation}/{league}-{season}/results/"
soup = BeautifulSoup(requests.get(url).content, "html.parser")

In [ ]:
# Webscraping do site BetExplorer

def get_odd_or_text(td):
    if "data-odd" in td.attrs:
        return td["data-odd"]

    odd = td.select_one("[data-odd]")
    if odd:
        return odd["data-odd"]

    return td.get_text(strip=True)

all_data = []
for row in soup.select(".table-main tr:has(td)"):
    tds = [get_odd_or_text(td) for td in row.select("td")]
    round_ = row.find_previous("th").find_previous("tr").th.text
    all_data.append([round_, *tds])

In [ ]:
all_data

# Organizando e Salvando o DataFrame

In [ ]:
df = pd.DataFrame(all_data, columns=["Rodada", "Partida", "Placar", "Odd_H", "Odd_D", "Odd_A", "Date"])
df['League'] = league

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df['Home'] = [i.split('-')[0] for i in df['Partida']]
df['Away'] = [i.split('-')[1] for i in df['Partida']]
df

In [ ]:
df = df.iloc[::-1]
df

In [ ]:
df['Goals_H'] = [i.split(':', 1)[0] if len(i.split(':')) >= 2 else None for i in df['Placar']]
df['Goals_A'] = [i.split(':', 1)[1] if len(i.split(':')) >= 2 else None for i in df['Placar']]


In [ ]:
def Result(row):
    if row['Goals_H'] is not None and row['Goals_A'] is not None:
        if row['Goals_H'] > row['Goals_A']:
            return "H"
        elif row['Goals_H'] < row['Goals_A']:
            return "A"
        else:
            return "D"
    else:
        return None

df['Result'] = df.apply(Result, axis=1)

In [ ]:
# df.replace("1. Round", 1, inplace=True)

for i in range(38):
  i = i + 1
  df.replace(str(i)+". Round", i, inplace=True)

df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df

In [ ]:
df.drop(['Partida','Placar'],axis=1,inplace=True)
df

In [ ]:
df = df[['League','Date','Rodada','Home','Away','Odd_H','Odd_D','Odd_A','Goals_H','Goals_A','Result']]
df

# Gerando para Todas as Temporadas

In [ ]:
nation = "south-africa"
league = "premier-league"

for year in range(2021, 2024):
    url = f"https://www.betexplorer.com/soccer/{nation}/{league}-{year}-{year+1}/results/"
    soup = BeautifulSoup(requests.get(url).content, "html.parser")

    # Webscraping do site BetExplorer

    def get_odd_or_text(td):
        if "data-odd" in td.attrs:
            return td["data-odd"]

        odd = td.select_one("[data-odd]")
        if odd:
            return odd["data-odd"]

        return td.get_text(strip=True)

    all_data = []
    for row in soup.select(".table-main tr:has(td)"):
        tds = [get_odd_or_text(td) for td in row.select("td")]
        round_ = row.find_previous("th").find_previous("tr").th.text
        all_data.append([round_, *tds])

    df = pd.DataFrame(all_data, columns=["Rodada", "Partida", "Placar", "Odd_H", "Odd_D", "Odd_A", "Date"])
    df['League'] = league

    df['Home'] = [i.split('-')[0] for i in df['Partida']]
    df['Away'] = [i.split('-')[1] for i in df['Partida']]

    df = df.iloc[::-1]

    df['Goals_H'] = [i.split(':', 1)[0] if len(i.split(':')) >= 2 else None for i in df['Placar']]
    df['Goals_A'] = [i.split(':', 1)[1] if len(i.split(':')) >= 2 else None for i in df['Placar']]

    def Result(row):
        if row['Goals_H'] is not None and row['Goals_A'] is not None:
            if row['Goals_H'] > row['Goals_A']:
                return "H"
            elif row['Goals_H'] < row['Goals_A']:
                return "A"
            else:
                return "D"
        else:
            return None

    df['Result'] = df.apply(Result, axis=1)

    for i in range(38):
        i = i + 1
        df.replace(str(i)+". Round", i, inplace=True)

    df.reset_index(inplace=True, drop=True)
    df.index = df.index.set_names(['Nº'])
    df = df.rename(index=lambda x: x + 1)

    df.drop(['Partida', 'Placar'], axis=1, inplace=True)

    df = df[['League', 'Date', 'Rodada', 'Home', 'Away', 'Odd_H', 'Odd_D', 'Odd_A', 'Goals_H', 'Goals_A', 'Result']]

    df.to_excel(league+"_"+str(year)+"-"+str(year+1)+".xlsx")
